# Extracción de características HOG
Este notebook aplica el descriptor HOG (Histogram of Oriented Gradients) a las imágenes normalizadas

In [ ]:
import os
import glob
import numpy as np
import cv2
from skimage.feature import hog
from tqdm import tqdm
import matplotlib.pyplot as plt

plt.ioff()  # Evita mostrar figuras vacías

BASE_DIR = os.path.dirname(os.path.abspath('.'))         # .../ProyectoFinal/HOG
PROYECTO_DIR = os.path.dirname(BASE_DIR)                 # .../ProyectoFinal

NORMALIZADAS_DIR = os.path.join(PROYECTO_DIR, 'normalizacion', 'normalizadas')
HOG_DIR = os.path.join(PROYECTO_DIR, 'HOG', 'hog_npy')

# Crear carpeta de salida si no existe
os.makedirs(HOG_DIR, exist_ok=True)

print(f"Ruta de imágenes normalizadas: {NORMALIZADAS_DIR}")
print(f"Ruta de salida HOG: {HOG_DIR}")

# 2. Parámetros HOG
orientations = 9
pixels_per_cell = (8, 8)
cells_per_block = (2, 2)

# 3. Función de procesamiento
def procesar_imagenes(origen_split):
    split_path = os.path.join(NORMALIZADAS_DIR, origen_split)
    if not os.path.exists(split_path):
        print(f"❌ Carpeta no encontrada: {split_path}")
        return

    out_split_path = os.path.join(HOG_DIR, origen_split)
    os.makedirs(out_split_path, exist_ok=True)

    clases = [d for d in os.listdir(split_path)
              if os.path.isdir(os.path.join(split_path, d))]

    for clase in clases:
        clase_in_dir = os.path.join(split_path, clase)
        clase_out_dir = os.path.join(out_split_path, clase)
        os.makedirs(clase_out_dir, exist_ok=True)

        imagenes = glob.glob(os.path.join(clase_in_dir, '*_recorte.png'))
        if len(imagenes) == 0:
            print(f"⚠️ No se encontraron imágenes en {clase_in_dir}")
            continue

        for img_path in tqdm(imagenes, desc=f"Procesando {clase} en {origen_split}"):
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                print(f"⚠️ No se pudo leer la imagen: {img_path}")
                continue

            features = hog(
                img,
                orientations=orientations,
                pixels_per_cell=pixels_per_cell,
                cells_per_block=cells_per_block,
                visualize=False
            )

            nombre_base = os.path.basename(img_path).replace('_recorte.png', '')
            out_path = os.path.join(clase_out_dir, f"{nombre_base}_hog.npy")
            np.save(out_path, features)

# 4. Ejecutar procesamiento de train y test
for split in ['train', 'test']:
    procesar_imagenes(split)

plt.close('all')
print("\n✅ Procesamiento HOG completado")
print("Las imágenes HOG han sido guardadas en la carpeta hog_npy/")


Ruta de imágenes normalizadas: /workspaces/fruit-detector-pricer/ProyectoFinal/normalizacion/normalizadas
Ruta de salida HOG: /workspaces/fruit-detector-pricer/ProyectoFinal/HOG/hog_npy


Procesando Pineapple Mini 1 en test: 100%|██████████| 148/148 [00:00<00:00, 167.91it/s]


✅ Procesamiento HOG completado
Las imágenes HOG han sido guardadas en la carpeta hog_npy/


In [5]:
from pathlib import Path

def contar_npy_por_split(split):
    base = Path(HOG_DIR) / split
    print(f"\n📦 Conteo en {split.upper()}:")
    for clase in sorted(base.iterdir()):
        if clase.is_dir():
            count = len(list(clase.glob("*.npy")))
            print(f"  {clase.name}: {count} archivos")

contar_npy_por_split("train")
contar_npy_por_split("test")



📦 Conteo en TRAIN:
  Apple Red 1: 344 archivos
  Banana 1: 343 archivos
  Cantaloupe 2: 344 archivos
  Cocos 1: 343 archivos
  Granadilla 1: 342 archivos
  Kiwi 1: 326 archivos
  Maracuja 1: 343 archivos
  Peach 2: 516 archivos
  Pear Forelle 1: 490 archivos
  Pineapple Mini 1: 343 archivos

📦 Conteo en TEST:
  Apple Red 1: 148 archivos
  Banana 1: 147 archivos
  Cantaloupe 2: 148 archivos
  Cocos 1: 147 archivos
  Granadilla 1: 147 archivos
  Kiwi 1: 140 archivos
  Maracuja 1: 147 archivos
  Peach 2: 222 archivos
  Pear Forelle 1: 211 archivos
  Pineapple Mini 1: 148 archivos


In [6]:
def total_npy(split):
    base = Path(HOG_DIR) / split
    total = sum(len(list(d.glob("*.npy"))) for d in base.iterdir() if d.is_dir())
    print(f"\n🔢 Total archivos en {split.upper()}: {total}")

total_npy("train")
total_npy("test")



🔢 Total archivos en TRAIN: 3734

🔢 Total archivos en TEST: 1605
